In [29]:
import glob
import os

def find_files(folder_path, expression):
    """
    Finds all files in the specified folder that match the given expression.

    Args:
        folder_path (str): The path to the folder to search.
        expression (str): A glob expression to match file names.

    Returns:
        list[str]: A list of file paths.
    """

    files = glob.glob(os.path.join(folder_path, expression))
    return files


In [30]:
import json
import gzip
import pysam

def process_bed_bam_pairs(bed_files, bam_files):
    """
    Processes pairs of BED and BAM files, calculating feature counts for each pair and writing the results to JSON files.

    Args:
        bed_files (list[str]): A list of BED file paths.
        bam_files (list[str]): A list of BAM file paths.

    Returns:
        None
    """

    for bed_file, bam_file in zip(bed_files, bam_files):
        # Process the bed and bam file pair and get the feature counts
        feature_counts = process_bed_bam_pair(bed_file, bam_file)

        # Create the JSON filename by combining the bed and bam filenames
        json_filename = f"{os.path.basename(bed_file)}_{os.path.basename(bam_file)}.json"

        # Prepare JSON data for the current bed-bam pair
        json_data = {
            "bed_file": bed_file,
            "bam_file": bam_file,
            "feature_counts": feature_counts
        }

        # Write the combined JSON data to a single JSON file
        with open(json_filename, "w") as json_file:
            json.dump(json_data, json_file, indent=4)

In [31]:
def process_bed_bam_pair(bed_file, bam_file):
    """
    Calculates feature counts for a given pair of BED and BAM files.

    Args:
        bed_file (str): The path to the BED file containing genomic features.
        bam_file (str): The path to the BAM file containing aligned reads.

    Returns:
        dict[str, int]: A dictionary mapping feature keys to their respective counts.
    """

    # Open the bed file and bam file
    with gzip.open(bed_file, 'rt') as bed_handle, pysam.Samfile(bam_file, 'rb') as bam_handle:
        # Create a dictionary to store feature counts
        feature_counts = {}

        # Iterate over the features in the bed file
        for line in bed_handle:
            # Extract the feature information from the bed line
            chrom, start, end = line.strip().split('\t')

            # Create a key for the feature
            feature_key = f"{chrom}:{start}-{end}"

            # Initialize the count for the feature
            feature_counts.setdefault(feature_key, 0)

            # Iterate over the reads in the bam file that overlap the feature
            for read in bam_handle.fetch(chrom, int(start), int(end)):
                # Increment the count for the feature
                feature_counts[feature_key] += 1

    return feature_counts

In [32]:
#Main script
#Step 1 get files in folder with certain expressions
bam_files = find_files("../data/input/","*.bam")
bed_files = find_files("../data/input/","*.bed*")
sorted_bam_files = []

#Step 2: Go over bam files sorting and indexing. 
for file in bam_files:
    output_folder = "../data/processed/"
    pysam.sort("-o",os.path.join(output_folder,f"sorted_{os.path.basename(file)}"),file)
    sorted_bam_files.append(os.path.join(output_folder,f"sorted_{os.path.basename(file)}"))
    pysam.index("-o",os.path.join(output_folder,f"sorted_{os.path.basename(file)}.bai"), os.path.join(output_folder,f"sorted_{os.path.basename(file)}"))

# Process all bed-bam file pairs
process_bed_bam_pairs(bed_files, sorted_bam_files)
